In [382]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools

### Data Preparation (fake data)

In [387]:
np.random.seed(0)

columns = ['Height', 'Weight']
index = ['person_' + str(x) for x in range(1, 11)]
height_val = [np.random.randint(160, 200) for x in index]
weight_val = [np.random.randint(60, 100) for x in index]
values = zip(height_val, weight_val)

df = pd.DataFrame(values, index=index, columns=columns)
df = df.reset_index()
df['BMI'] = ((df['Weight'] / df['Height']**2 * 10e3) > 25).astype(int)
df

,index,Height,Weight,BMI
0,person_1,160,84,1
1,person_2,163,84,1
2,person_3,163,72,1
3,person_4,199,61,0
4,person_5,169,98,1
5,person_6,179,99,1
6,person_7,181,83,1
7,person_8,196,84,0
8,person_9,183,77,0
9,person_10,166,97,1


In [388]:
df.drop('index', axis=1, inplace=True)
df = df.astype(float)
df

,Height,Weight,BMI
0,160.0,84.0,1.0
1,163.0,84.0,1.0
2,163.0,72.0,1.0
3,199.0,61.0,0.0
4,169.0,98.0,1.0
5,179.0,99.0,1.0
6,181.0,83.0,1.0
7,196.0,84.0,0.0
8,183.0,77.0,0.0
9,166.0,97.0,1.0


In [392]:
names = df.columns[:2]
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df.iloc[:,:2])
scaled_df = pd.DataFrame(scaled_df, columns=names)
scaled_df['BMI'] = df['BMI']
scaled_df

,Height,Weight,BMI
0,0.000000,0.605263,1.0
1,0.076923,0.605263,1.0
2,0.076923,0.289474,1.0
3,1.000000,0.000000,0.0
4,0.230769,0.973684,1.0
5,0.487179,1.000000,1.0
6,0.538462,0.578947,1.0
7,0.923077,0.605263,0.0
8,0.589744,0.421053,0.0
9,0.153846,0.947368,1.0


In [397]:
scaled_df_ts = scaled_df.transpose()
scaled_df_ts

,0,1,2,3,4,5,6,7,8,9
Height,0.000000,0.076923,0.076923,1.0,0.230769,0.487179,0.538462,0.923077,0.589744,0.153846
Weight,0.605263,0.605263,0.289474,0.0,0.973684,1.000000,0.578947,0.605263,0.421053,0.947368
BMI,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000


In [394]:
x = scaled_df.iloc[:8,:2].values.transpose()
y = scaled_df.iloc[:8,2:].values.transpose()

xval = scaled_df.iloc[8:,:2].values.transpose()
yval = scaled_df.iloc[8:,2:].values.transpose()

x, x.shape, y, y.shape

(array([[0.        , 0.07692308, 0.07692308, 1.        , 0.23076923,
         0.48717949, 0.53846154, 0.92307692],
        [0.60526316, 0.60526316, 0.28947368, 0.        , 0.97368421,
         1.        , 0.57894737, 0.60526316]]),
 (2, 8),
 array([[1., 1., 1., 0., 1., 1., 1., 0.]]),
 (1, 8))

### Creating Neural Networl Class

In [458]:
def Sigmoid(Z):
    return 1/(1+np.exp(-Z))

def Relu(Z):
    return np.maximum(0,Z)

class Dlnet(object):
    def __init__(self, x, y):
        self.X = x
        self.Y = y
        self.Yh = np.zeros((1, self.Y.shape[1]))
        
        self.dims = [2,3,1]
        
        self.param = {}
        self.ch = {}
        
    def ninit(self):
        np.random.seed(1)
        self.param['W1'] = np.random.randn(self.dims[1], self.dims[0]) / np.sqrt(self.dims[0])
        self.param['b1'] = np.zeros((self.dims[1],1))
        self.param['W2'] = np.random.randn(self.dims[2], self.dims[1]) / np.sqrt(self.dims[1])
        self.param['b2'] = np.zeros((self.dims[2],1))
        return
   
    def nloss(self,Yh):
        loss = (1./self.sam) * (-np.dot(self.Y,np.log(Yh).T) - np.dot(1-self.Y, np.log(1-Yh).T))    
        return loss
    
    def forward(self):    
        Z1 = self.param['W1'].dot(self.X) + self.param['b1'] 
        A1 = Relu(Z1)
        self.ch['Z1'],self.ch['A1']=Z1,A1
        
        Z2 = self.param['W2'].dot(A1) + self.param['b2']  
        A2 = Sigmoid(Z2)
        self.ch['Z2'],self.ch['A2']=Z2,A2

        self.Yh=A2
        #loss=self.nloss(A2)
        return self.Yh, #loss

In [459]:
x_ = x[:, :1]
y_ = y[:, :1]
x_, x_.shape, y_, y_.shape

(array([[0.        ],
        [0.60526316]]), (2, 1), array([[1.]]), (1, 1))

In [460]:
nn = Dlnet(x_, y_)
nn.ninit()
nn.forward()

(array([[0.5]]),)